In [16]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import pandas as pd
import numpy as np
import os

In [17]:
dash_line = '-' * 80

In [18]:
with open("game-data.txt", "r") as file:
    data = file.read()

print(type(data))

<class 'str'>


In [19]:
import re

# Split the game data into separate strings at each 'Game Over' line
game_data_parts = re.split('(\[Game Over\]:.*\n)', data)

# Combine every two parts (narrative and game over)
game_data_parts = [game_data_parts[n] + game_data_parts[n + 1] for n in range(0, len(game_data_parts) - 1, 2)]

print(str(len(game_data_parts)) + '\n' + dash_line + '\n' + (game_data_parts[-1]))

113
--------------------------------------------------------------------------------

[Narrator]: As you journey deeper into the enchanted forest, you stumble upon a nest of giant spiders, their hairy legs skittering across the forest floor as they prepare to attack.

[Player]: Confront the spiders!

[Narrator]: With nerves of steel, you face the spiders head-on, ready to defend yourself against their venomous bites. As the first spider lunges towards you, you brace yourself for the coming battle.

[Player]: Dodge the spiders' attacks!

[Narrator]: With lightning-fast reflexes, you evade the spiders' strikes, leaping out of harm's way and rolling to safety. As you maneuver around the creatures, you search for an opening to strike back.

[Player]: Attack the spiders' vulnerable spots!

[Narrator]: Spotting a weakness in the spiders' armor, you deliver a series of precise strikes, aiming for their soft underbellies. With each blow, you weaken the spiders' defenses, until at last, they li

In [20]:
for i in range(100):
    print(game_data_parts[i+100])
    print(dash_line)


[Narrator]: Amidst the ancient trees of the enchanted forest, you encounter a pack of ferocious wolves, their eyes gleaming with hunger as they circle around you.

[Player]: Prepare to defend against the wolves.

[Narrator]: With your back against a tree, you ready your weapon, steeling yourself for the coming onslaught. As the wolves close in, you stand firm, ready to fight for your survival.

[Player]: Ward off the wolves' attacks.

[Narrator]: As the wolves lunge at you with bared teeth, you fend them off with quick strikes and well-timed blocks. Despite their relentless assault, you refuse to back down, determined to protect yourself at all costs.

[Player]: Fight back against the wolves.

[Narrator]: Seizing an opportunity, you unleash a flurry of attacks, driving the wolves back with each powerful blow. With a final, decisive strike, you scatter the pack, sending them fleeing into the depths of the forest.

[Game Over]: In the heart of the enchanted forest, you have emerged vict

IndexError: list index out of range

In [ ]:
longest_strings = sorted(game_data_parts, key=len, reverse=True)[:2]
print(longest_strings)
print(len(longest_strings[0]), len(longest_strings[1]))

['[Narrator]: As you venture deeper into the enchanted forest, the dense canopy above casts shifting patterns of light and shadow upon the forest floor. You can feel the mystical energy pulsating around you, hinting at the presence of fantastical beings lurking amidst the trees. Suddenly, you come across a clearing where a majestic creature stands before you, its eyes gleaming with an otherworldly intelligence.\n\n[Player]: I cautiously approach the creature and observe its demeanor.\n\n[Narrator]: The creature, resembling a graceful unicorn with a shimmering coat of silver, regards you with curiosity. Its long mane sways gently in the breeze as it tilts its head, seemingly assessing you.\n\n[Player]: I extend my hand slowly, offering it for the creature to sniff, attempting to establish a peaceful interaction.\n\n[Narrator]: The unicorn lowers its head, its muzzle brushing against your outstretched hand. For a moment, there is a tense silence, then the creature lets out a soft whinny 

In [21]:
for i, game_data_part in enumerate(game_data_parts):
    with open(f"data/game_data_{i+1}.txt", "w") as file:
        print(len(game_data_part))
        file.write(game_data_part)

12073
2514
2098
2501
2319
1967
2350
2312
2121
1709
2204
2191
2788
2847
2879
2806
2874
1947
1964
1967
1954
908
980
1012
921
1037
1108
969
1074
1091
1103
1061
1563
1473
1638
1641
1618
1628
1812
1953
2065
1886
2744
1695
1145
1086
1204
1245
1227
1393
1371
1435
2310
2577
2612
2635
2335
2072
2620
2058
1811
2037
1757
1757
2002
2420
2394
2036
2079
2079
1662
1662
1319
1272
841
515
506
576
478
1454
1584
1734
2133
2023
2079
2179
2307
1445
1600
1221
1263
1450
1320
1311
1341
1256
1329
1322
1232
1201
1231
1301
1309
1251
1701
1275
1300
1120
1151
1259
1188
1256
1217


In [9]:
data_directory = "data"
file_names = os.listdir(data_directory)
# file_names.remove("game-data.txt")
print(len(file_names))

114


In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map = 'auto')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [121]:
from datasets import Dataset, DatasetDict

data_directory = "data"

datasets = DatasetDict()
for file in file_names:
    with open(f"{data_directory}/{file}", 'r') as f:
        data = f.read()
    datasets[file] = Dataset.from_dict({'text': [data]})

In [122]:
len(datasets)

39

In [123]:
print(datasets['game_data_5.txt']['text'][0])


[Narrator]: As you traverse deeper into the heart of the enchanted forest, the air crackles with an aura of mystique, hinting at the presence of powerful beings lurking amidst the shadows. Suddenly, you stumble upon a clearing where a fearsome creature stands before you, its massive form towering over the forest floor. With razor-sharp claws and gleaming fangs, it regards you with a mixture of curiosity and hostility.

[Player]: I approach the creature slowly, keeping my movements calm and non-threatening as I attempt to establish a connection.

[Narrator]: With cautious steps, you approach the fearsome beast, your heart pounding in your chest as you meet its piercing gaze. Sensing your intent, the creature's demeanor shifts, its hostility giving way to cautious interest. With a steady hand, you reach out, offering a gesture of friendship in the hopes of earning its trust.

[Player]: I speak softly to the creature, offering soothing words and gentle gestures as I seek to tame its wild

In [124]:
def tokenize_function(example):
    start_prompt = '[Game Start]\n\n'
    example['text'] = tokenizer(start_prompt + example['text'])
    return example

In [125]:
datasets = datasets.map(tokenize_function)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [134]:
print(datasets['game_data_20.txt']['text'][0]['input_ids'])

[2, 235309, 6242, 7248, 235307, 110, 235309, 154506, 8254, 101250, 490, 573, 12387, 8195, 576, 573, 115885, 9309, 235269, 1570, 33812, 65856, 71548, 12136, 578, 573, 2681, 84976, 5993, 675, 193124, 4134, 235269, 476, 58760, 27988, 81676, 1560, 235265, 9707, 4628, 235269, 134775, 675, 120610, 57868, 235269, 6530, 607, 3054, 692, 685, 665, 73643, 774, 573, 22945, 235269, 1277, 15665, 1736, 31864, 476, 146103, 12425, 1163, 573, 9309, 6784, 235265, 109, 235309, 7324, 8254, 3279, 17564, 575, 970, 3760, 235269, 590, 5628, 970, 17175, 235269, 8598, 577, 30538, 573, 27988, 578, 14429, 6836, 2691, 1277, 53762, 235265, 109, 235309, 154506, 8254, 714, 9309, 1004, 3374, 675, 573, 43617, 576, 8212, 685, 692, 18254, 575, 476, 44597, 9509, 675, 573, 27988, 235269, 1853, 15657, 476, 36863, 8697, 577, 114617, 1401, 1277, 104747, 25290, 235265, 24857, 861, 9331, 235269, 573, 30365, 38691, 577, 614, 476, 82056, 101071, 235269, 1277, 18812, 30051, 578, 75575, 235265, 109, 235309, 7324, 8254, 590, 5900, 61

In [136]:
output_dir = f'./model-training-{str(time.time())}'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets,
)

RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.